<a href="https://colab.research.google.com/github/pranav-vijayananth/SOMResearch/blob/main/pranav_keras_som.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#KERAS-SOM MAIN MODULE
!wget https://raw.githubusercontent.com/FlorentF9/DESOM/master/kerasom.py

#SOM-LAYER DEPENDENCY FOR MAIN MODULE
!wget https://raw.githubusercontent.com/FlorentF9/DESOM/master/SOM.py

#METRICS 
!wget https://raw.githubusercontent.com/FlorentF9/DESOM/master/metrics.py

#DATASETS
!wget https://raw.githubusercontent.com/FlorentF9/DESOM/master/datasets.py

In [ ]:
#IMPORTS

from kerasom import Kerasom
import numpy as np
import math 
import subprocess
from time import time
from collections import Counter

In [ ]:
#GETTING FILE

# filename = "breast-tissue.csv"
datafile = open("/content/drive/MyDrive/Datasets/commonfiles/breast-cancer.csv")

In [ ]:
#DATA PREP 

data = []
groundtruth = []
dataread = datafile.readline()
dataread = datafile.readline()

while dataread != "": 
  a = dataread.split(",")
  l2 = []
  for j in range(0, len(a), 1):
    if j == len(a)-1:
      groundtruth.append(a[j].strip())
    else:
      try: 
        l2.append(float(a[j]))
      except:
        l2.append(0)
  data.append(l2)
  dataread = datafile.readline()

rows = len(data)
cols = len(data[0])
num_inputs = rows * cols

datafile.close()

In [ ]:
#UNDERSTANDING DATA 
print(f"This is the data: {data}")
print(f"Number of rows: {rows}")
print(f"Numer of columns: {cols}")
print(f"Total number of data inputs: {num_inputs}")

#CLUSTERS 
uniqueValues = Counter(groundtruth).keys()
clusters = len(uniqueValues)
print(f"The clusters in dataset: {clusters}")

In [ ]:
#MAKING TXT FILE FOR LABELS

# bash_command = "touch /content/drive/MyDrive/Datasets/Research/Keras-SOM/"+filename+".txt"
# subprocess.run(bash_command, shell=True)

# fw = open("/content/drive/MyDrive/Datasets/Research/Keras-SOM/"+filename+".txt", "a")

#DATA 
data = np.asarray(data)

#NEURONS 
mapsize = (clusters, 1)

"""
Change the map_size to equal to [clusters, 1] for ARI labels later
"""

#OPTIMIZER ß
optimizer = 'adam'

#BATCH SIZE
batch_size = 32

#ITERATIONS 
iterations = int(rows/batch_size)

for i in range(30):
  #MODEL INIT + COMPLIE 
  kerasom = Kerasom(input_dim=cols, map_size=(clusters, 1))
  kerasom.initialize()
  kerasom.compile(optimizer=optimizer)

  #MODEL TRAINING
  # start = time()
  kerasom.fit(data, iterations=iterations, som_iterations=iterations, save_epochs=20, batch_size=batch_size, save_dir="/content/drive/MyDrive/Datasets/Research/Keras-SOM/Logs")

  #GETTING PREDICTIONS
  predictions = kerasom.predict(data)
  print(predictions)

  #ADDING LABELS INTO TXT FILE
#   converted_list = [str(element) for element in predictions]
#   joined_string = ",".join(converted_list)
#   print(joined_string)
#   fw.write(joined_string)
#   fw.write('\n')
#   print(i)

# fw.close()

In [ ]:
#OPENING FILE FOR ARI 

fo = open("/content/drive/MyDrive/Datasets/Research/Keras-SOM/"+filename+".txt", "r")
fw = open("/content/drive/MyDrive/Datasets/Research/Keras-SOM/breast-tissue_output.csv", 'a')

In [ ]:
#ADDING ARI TO FILES

count = 1 
fw.write(str(filename))
for line in fo: 
  line = line.strip()
  line = line.strip("\n")
  curlabel = line.split(",")
  if count > 1:
    ARI = adjusted_rand_score(prevlabel, curlabel)
    print(ARI)
    fw.write(','+str(ARI))
  
  prevlabel = curlabel
  count = 2

fw.write("\n")
fw.close()